In [1]:
import sys
sys.path.append('..')
from QCreator import transmission_line_simulator as tlsim
from QCreator import conformal_mapping as cm
import pandas as pd
from scipy.constants import hbar, e, epsilon_0, mu_0
import numpy as np

In [2]:
#50 Ohm line
cl_tl, Ll_tl = cm.ConformalMapping([6, 10, 6]).cl_and_Ll()

In [3]:
Ec = 330e6
Ecc = 250e6
f1 = 6.143e9
f2 = 6.421e9

fbus = 7.063e9
fcmax = 7.1e9
epsilon = 11.45

ccp = 2.5e-15
ccm = 0.8e-15

lbus = (np.sqrt(2/(epsilon_0*mu_0*(epsilon+1)))/(2*fbus) - 2*ccp/cl_tl[0,0]/1e6)
cq_full = e**2/(2*Ec*2*np.pi*hbar)
cc_full = e**2/(2*Ecc*2*np.pi*hbar)
Ej1 = (f1 + Ec)**2/(8*Ec)
Ej2 = (f2 + Ec)**2/(8*Ec)
Ejcmax = (fcmax + Ecc)**2/(8*Ecc)

print ('cq_full [fF]: ', cq_full/1e-15)
print ('cc_full [fF]: ', cc_full/1e-15)
print ('bus length [um]: ', lbus/1e-6)
print ('Ej1 [GHz]: ', Ej1/1e9)
print ('Ej2 [GHz]: ', Ej2/1e9)
print ('Ejcmax [GHz]: ', Ejcmax/1e9)

cq_full [fF]:  58.69766462017915
cc_full [fF]:  77.48091729863648
bus length [um]:  8475.544785594362
Ej1 [GHz]:  15.871109469696972
Ej2 [GHz]:  17.26363674242424
Ejcmax [GHz]:  27.01125


In [4]:


tls = tlsim.TLSystem()

q1c = tlsim.Capacitor(c=cq_full-ccp-ccm, name='q1c')
q2c = tlsim.Capacitor(c=cq_full-ccp-ccm, name='q2c')
cmc = tlsim.Capacitor(c=cc_full-ccm, name='q2c')

jj1 = tlsim.JosephsonJunction(e_j=Ej1*hbar*2*np.pi, name='jj1')
jj2 = tlsim.JosephsonJunction(e_j=Ej2*hbar*2*np.pi, name='jj2')
jjcmax = tlsim.JosephsonJunction(e_j=Ejcmax*hbar*2*np.pi, name='jjcmax')

cptl = tlsim.TLCoupler(n=1, l=lbus*1e6, ll=Ll_tl, cl=cl_tl, 
                      rl=np.zeros((1,1)), gl=np.zeros((1,1)), name='cptl')

cpc1 = tlsim.Capacitor(c=ccp, name='cpc1')
cpc2 = tlsim.Capacitor(c=ccp, name='cpc2')

cmc1 = tlsim.Capacitor(c=ccm, name='cmc1')
cmc2 = tlsim.Capacitor(c=ccm, name='cmc2')

tls.add_element(tlsim.Short(), [0])
tls.add_element(q1c, [0, 1])
tls.add_element(q2c, [0, 2])
tls.add_element(jj1, [0, 1])
tls.add_element(jj2, [0, 2])
tls.add_element(cpc1, [1, 3])
tls.add_element(cpc2, [2, 4])
tls.add_element(cptl, [3, 4])
tls.add_element(cmc1, [1, 5])
tls.add_element(cmc2, [2, 6])
tls.add_element(cmc, [5, 6])
tls.add_element(jjcmax, [5, 6])

In [5]:
omega, delta, modes = tls.get_modes()

In [6]:
omega/(2*np.pi)

array([6.48973891e+09, 6.77070480e+09, 7.07257676e+09, 7.37045613e+09,
       1.41607162e+10, 1.91130721e+10, 2.21272635e+10, 2.21272635e+10,
       2.23217285e+10, 2.40681593e+10, 2.40681593e+10, 2.31038662e+11])

In [7]:
pd.DataFrame(tls.get_perturbation([0,1,2,3]))

,0,1,2,3
0,-3.215695e+08,-1.235730e+07,-3.185101e+06,-6.805041e+05
1,-1.235730e+07,-3.135164e+08,-1.243561e+07,-1.302265e+06
2,-3.185101e+06,-1.243561e+07,-1.300480e+05,-1.344053e+05
3,-6.805041e+05,-1.302265e+06,-1.344053e+05,-2.504724e+08


In [8]:
omega, kerr = tls.get_perturbation_hamiltonian_kerr([0,1], [5,5])
omega, kerr

(array([6.12774002e+09, 6.42044034e+09]),
 array([[-4.09896072e+08,  6.98223528e+06],
        [ 6.98223528e+06, -4.02896277e+08]]))

In [9]:
pd.DataFrame(tls.get_second_order_perturbation_kerr2([0,1]))

..\QCreator\transmission_line_simulator.py:1411: RuntimeWarning: divide by zero encountered in double_scalars
  correction_new = numerator / denominator


,0,1
0,-4.014390e+08,-7.815633e+06
1,-7.815633e+06,-3.727213e+08


In [10]:
pd.DataFrame(tls.get_second_order_perturbation_kerr2([0,1], raw_frequencies=1))

..\QCreator\transmission_line_simulator.py:1411: RuntimeWarning: divide by zero encountered in double_scalars
  correction_new = numerator / denominator


,0,1
0,-4.128208e+08,5.418914e+07
1,5.418914e+07,-4.552941e+08


In [ ]:
E00 = tls.get_second_order_perturbation([0,0], [0,1])
E01 = tls.get_second_order_perturbation([0,1], [0,1])
E10 = tls.get_second_order_perturbation([1,0], [0,1])
E11 = tls.get_second_order_perturbation([1,1], [0,1])
E11 + E00 - E01 - E10

# rubbish

In [10]:
H, basis = tls.get_perturbation_hamiltonian([0,1], [3,3])
H.shape

(9, 9)

In [10]:
np.sum(np.abs(H-np.conj(H).T))/np.sum(np.abs(H))

NameError: name 'H' is not defined

In [12]:
energies, vecs = np.linalg.eigh(H)

In [13]:
energies

array([-1.51443087e+08,  4.83785629e+09,  5.03079668e+09,  9.54267206e+09,
        9.92003743e+09,  1.00262244e+10,  1.47119059e+10,  1.49046753e+10,
        1.95960233e+10])

In [21]:
np.round(np.abs(vecs[1,:]),2)

array([0.  , 0.01, 1.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.  ])

In [22]:
np.conj(vecs[:,1])@H@vecs[:,1]

(4837856288.900802-6.938893903907228e-18j)

In [ ]:
omega

In [ ]:
kerr

In [ ]:
e